# Phase Field Benchmark 8d
## Athermal nulceation
FiPy implementation of problem 2.4 in *Benchmark problems for nucleation*, Tamás Pusztai, September 25, 2019

**Do not edit `benchmark8d.py`**. Generate the batch-runnable file from the notebook with
```bash
jupyter nbconvert benchmark8d.ipynb --to python
```

## Import Python modules

In [ ]:
import os
import re
import sys
import yaml

import datreant.core as dtr

import fipy as fp
from fipy.tools import parallelComm
from fipy.meshes.factoryMeshes import _dnl

from fipy.tools.debug import PRINT

Jupyter notebook handles some things differently than from the commandline

In [ ]:
try:
    from IPython import get_ipython
    isnotebook = get_ipython() is not None
except:
    isnotebook = False

## Initialize
### Load parameters

In [ ]:
if isnotebook:
    yamlfile = "params8d.yaml"
else:
    yamlfile = sys.argv[1]

with open(yamlfile, 'r') as f:
    if hasattr(yaml, "FullLoader"):
        # PyYAML 5.1 deprecated the plain yaml.load(input) function
        # https://github.com/yaml/pyyaml/wiki/PyYAML-yaml.load(input)-Deprecation
        params = yaml.load(f, Loader=yaml.FullLoader)
    else:
        params = yaml.load(f)

### Set any parameters for interactive notebook

In [ ]:
if isnotebook:
    params['dt'] = 10.0
    params['checkpoint_interval'] = 10. * params['dt']
    params['factor'] = 1.02
#    params['totaltime'] = 100 * params['dt']

### Initialize mesh and solution variables

Either restart from some `path/to/t={time}.tar.gz`, where the time is assigned to `elapsed`

or

Create a mesh based on parameters.
> Use a simulation domain of width 40 and height 20. Use Dirichlet boundary conditions on the bottom side by setting $\phi = 0.9$ along the middle part of length 20 of the boundary and $\phi = 0$ outside.

In [ ]:
checkpoint_interval = params['checkpoint_interval']
totaltime = params['totaltime']
dt = params['dt']

if params['restart']:
    phi, = fp.tools.dump.read(filename=params['restart'])
    mesh = phi.mesh

    X, Y = mesh.faceCenters

    Lx = mesh.communicator.MaxAll(max(X)) - mesh.communicator.MinAll(min(X))
    Ly = mesh.communicator.MaxAll(max(Y)) - mesh.communicator.MinAll(min(Y))

    # scanf("%g") simulator
    # https://docs.python.org/3/library/re.html#simulating-scanf
    scanf_g = "[-+]?(\d+(\.\d*)?|\.\d+)([eE][-+]?\d+)?"
    pattern = ".*t=({g})\.tar\.gz".format(g=scanf_g)
    elapsed = re.match(pattern, params['restart']).group(1)
    elapsed = fp.Variable(name="$t$", value=float(elapsed))
else:
    Lx = params['Lx']
    Ly = params['Ly']

    dx, nx = _dnl(dx=params['dx'], nx=None, Lx=Lx)
    dy, ny = _dnl(dx=params['dx'], nx=None, Lx=Ly)

    mesh = fp.PeriodicGrid2DLeftRight(dx=dx, nx=nx, dy=dy, ny=ny)

    phi = fp.CellVariable(mesh=mesh, name="$\phi$", value=0., hasOld=True)

    elapsed = fp.Variable(name="$t$", value=0.)
    
x, y = mesh.cellCenters[0], mesh.cellCenters[1]
X, Y = mesh.faceCenters[0], mesh.faceCenters[1]

phi.constrain(0.9, where=(mesh.facesBottom 
                          & (X <= Lx / 2 + params["radius"]) 
                          & (X >= Lx / 2 - params["radius"])))
phi.constrain(0.0, where=(mesh.facesBottom 
                          & ((X > Lx / 2 + params["radius"]) 
                             | (X < Lx / 2 - params["radius"]))))

In [ ]:
if isnotebook:
    viewer = fp.Viewer(vars=phi, datamin=0., datamax=1.)
    viewer.plot()

## Define governing equation

> use only the nondimensional forms of the phase-field and nucleation equations, but without the tildes, for simplicity

> Set the undercooling to $\Delta f_0 = 1 / (30\sqrt{2})$, which corresponds to the critical radius $r^* = 10$.

> Repeat the simulation with increasing/decreasing the driving force by 2%

In [ ]:
Delta_f = params['factor'] * 1. / (30 * fp.numerix.sqrt(2.))

and define the governing equation 
> \begin{align}
\frac{\partial\phi}{\partial t} &= \nabla^2\phi - g'(\phi) + \Delta f p'(\phi) \tag{7}
\end{align}

> $$g(\phi) = \phi^2(1 - \phi)^2 \qquad p(\phi)=\phi^3(10 - 15\phi + 6\phi^2)$$

Following [`examples/phase/simple.py`](https://www.ctcms.nist.gov/fipy/examples/phase/generated/examples.phase.simple.html)


\begin{align}
\frac{\partial\phi}{\partial t}
&= \nabla^2\phi + m_\phi \phi (1 - \phi) \notag
\qquad\text{for $m_\phi \equiv -2(1 - 2\phi) + 30 \phi (1 - \phi) \Delta f$} \notag
\\
&= \nabla^2\phi + S \notag
\\
&\approx \nabla^2\phi + S|_\text{old}
+ \left.{\frac{\partial S}{\partial \phi}}\right|_\text{old} (\phi - \phi_\text{old}) 
\notag \\
&= \nabla^2\phi + \left(S - \frac{\partial S}{\partial \phi} \phi\right)_\text{old} 
- \left.{\frac{\partial S}{\partial \phi}}\right|_\text{old} \phi \notag
\\
&= \nabla^2\phi + S_0 + S_1 \phi \notag
\\
S_0 &\equiv \left(S - \frac{\partial S}{\partial \phi} \phi\right)_\text{old}
\notag \\
&= m_\phi \phi_\text{old} (1 - \phi_\text{old}) - S_1 \phi_\text{old}
\notag \\
S_1 &\equiv \left.{\frac{\partial S}{\partial \phi}}\right|_\text{old}
\notag \\
&= \frac{\partial m_\phi}{\partial \phi} \phi (1 - \phi) + m_\phi (1 - 2\phi)
\notag
\end{align}

In [ ]:
mPhi = -2 * (1 - 2 * phi) + 30 * phi * (1 - phi) * Delta_f
dmPhidPhi = 4 + 30 * (1 - 2 * phi) * Delta_f
S1 = dmPhidPhi * phi * (1 - phi) + mPhi * (1 - 2 * phi)
S0 = mPhi * phi * (1 - phi) - S1 * phi

eq = (fp.TransientTerm() == 
      fp.DiffusionTerm(coeff=1.) + S0 + fp.ImplicitSourceTerm(coeff=S1))

## Calculate total free energy

> \begin{align}
F[\phi] = \int\left[\frac{1}{2}(\nabla\phi)^2 + g(\phi) - \Delta f p(\phi)\right]\,dV \tag{6}
\end{align}

In [ ]:
ftot = (0.5 * phi.grad.mag**2
        + phi**2 * (1 - phi)**2
        - Delta_f * phi**3 * (10 - 15 * phi + 6 * phi**2))
volumes = fp.CellVariable(mesh=mesh, value=mesh.cellVolumes)
F = ftot.cellVolumeAverage * volumes.sum()

## Setup output

### Setup ouput storage

In [ ]:
try:
    from sumatra.projects import load_project
    project = load_project(os.getcwd())
    record = project.get_record(params["sumatra_label"])
    output = record.datastore.root
except:
    # either there's no sumatra, no sumatra project, or no sumatra_label
    # this will be the case if this script is run directly
    output = os.getcwd()
    
if parallelComm.procID == 0:
    print "storing results in {0}".format(output)
    data = dtr.Treant(output)
else:
    class dummyTreant(object):
        categories = dict()

    data = dummyTreant()

### Define output routines

In [ ]:
def saveStats(elapsed):
    if parallelComm.procID == 0:
        fname = data['stats.txt'].make().abspath
        if os.path.exists(fname):
            # backup before overwrite
            os.rename(fname, fname + ".save")
        try:
            fp.numerix.savetxt(fname, 
                               stats, 
                               delimiter="\t", 
                               header="\t".join(["time", "fraction", "energy"]))
        except:
            # restore from backup
            os.rename(fname + ".save", fname)
        if os.path.exists(fname + ".save"):
            os.remove(fname + ".save")

def current_stats(elapsed):
    return [float(x) for x in [elapsed, phi.cellVolumeAverage, F]]

def savePhi(elapsed):
    if parallelComm.procID == 0:
        fname = data["t={}.tar.gz".format(elapsed)].make().abspath
    else:
        fname = None
    fname = parallelComm.bcast(fname)

    fp.tools.dump.write((phi,), filename=fname)

def checkpoint(elapsed):
    saveStats(elapsed)
    savePhi(elapsed)

### Figure out when to save

In [ ]:
checkpoints = (fp.numerix.arange(int(elapsed / checkpoint_interval),
                                 int(totaltime / checkpoint_interval)) + 1) * checkpoint_interval
for sometime in [totaltime]:
    if sometime > elapsed and sometime not in checkpoints: 
        checkpoints = fp.tools.concatenate([checkpoints, [sometime]])
checkpoints.sort()

### Output initial condition

In [ ]:
if params['restart']:
    fname = os.path.join(os.path.dirname(params['restart']), "stats.txt")
    stats = fp.numerix.loadtxt(fname)
    stats = stats[stats[..., 0] <= elapsed].tolist()
else:
    stats = []
    stats.append(current_stats(elapsed))

checkpoint(elapsed)

## Solve and output

In [ ]:
for until in checkpoints:
    while elapsed.value < until:
        phi.updateOld()
        dt_until = (until - elapsed).value
        dt_save = dt
        if dt_until < dt:
            dt = dt_until
        for sweep in range(30):
            res = eq.sweep(var=phi, dt=dt)
#            print elapsed, dt, sweep, res
        elapsed.value = elapsed() + dt
        stats.append(current_stats(elapsed))
        dt = dt_save

    checkpoint(elapsed)

    if isnotebook:
        viewer.plot()